In [ ]:
import glob
import os
import shutil

from astropy import units as u
from astropy.io import fits
from astropy.nddata import Cutout2D
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd


# Determine where the visualization will be rendered
output_notebook()

plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

In [ ]:
## Define all Paths here: 
#biyearly_image = '/Users/cmartlin/Desktop/postflash_2022_work/2018-2019_test_01_26_2022_idle02ayq_flc.fits'
#current_image = '/Users/cmartlin/Desktop/postflash_2022_work/idle02ayq/idle02ayq_flc.fits'
#yearly_image = '/Users/cmartlin/Desktop/postflash_2022_work/2018_yearly_low_10_18_2022_idn011e6s_flc.fits'

#updated_yearly = glob.glob('/Users/cmartlin/Desktop/postflash_2022_work/*10_18*')
#science_pipeline = glob.glob('/Users/cmartlin/Desktop/postflash_2022_work/i*/i*flc.fits')

In [ ]:
## Define all Paths here: 
biyearly_image = '../../../../postflash_2022/postflash_2022_work/2022_ISR_testing_data/2018-2019_test_01_26_2022_idle02ayq_flc.fits'
current_image = '../../../../postflash_2022/postflash_2022_work/2022_ISR_testing_data/idle02ayq_flc.fits'
#yearly_image = '../../postflash_2022_work/2022_ISR_testing_data/2018_yearly_low_10_18_2022_idn011e6s_flc.fits'
yearly_image = '../../../../postflash_2022/postflash_2022_work/2022_ISR_testing_data/2019_yearly_low_10_18_2022_idle02ayq_flc.fits'

updated_yearly = glob.glob('../../../../postflash_2022/postflash_2022_work/2022_ISR_testing_data/*10_18*')
science_pipeline = glob.glob('../../../../postflash_2022/postflash_2022_work/2022_ISR_testing_data/i*flc.fits')


current_image_2013 = '../../../../postflash_2022/postflash_2022_work/2022_ISR_testing_data/ic1707bmq_flc.fits'
#yearly_image = '../../postflash_2022_work/2022_ISR_testing_data/2018_yearly_low_10_18_2022_idn011e6s_flc.fits'
yearly_image_2013 = '../../../../postflash_2022/postflash_2022_work/2022_ISR_testing_data/2013_yearly_low_10_18_2022_ic1707bmq_flc.fits'


In [ ]:
#Checking the testing data rootnames used in function:
#print(updated_yearly)
for f in updated_yearly: 
    print(f[-18:-9])
    
#Checking for year length - if this needs to change, update in function below: 
for f in updated_yearly: 
    #print(f[22:26])
    #print(f[48:52])
    print(f[69:73])
    fname_left = 69
    fname_right = 73

In [ ]:
#Check the science data rootnames used in function:
for f in science_pipeline: 
    print(f[-18:-9])

In [ ]:
def create_databases(input_data, positions_list, fname_left, fname_right, db_return):
    size = (101, 101)
    
    min_pos = []
    max_pos = []
    mean_pos = []
    median_pos = []
    std_pos = []
    
    for f in input_data: 
        image_data = fits.getdata(f,1)
        means = []
        maxes = []
        mins = []
        medians = []
        stdevs = []
        for p in positions_list: 
            cutout = Cutout2D(image_data, p, size)
            cutout1 = cutout.data
            mins.append(np.min(cutout1))
            maxes.append(np.max(cutout1))
            means.append(np.mean(cutout1))
            medians.append(np.median(cutout1))
            stdevs.append(np.std(cutout1))
        std_pos.append(stdevs)
        min_pos.append(mins)
        max_pos.append(maxes)
        mean_pos.append(means)
        median_pos.append(medians)
    filenames = []
    years = []

    for f in input_data:
        filenames.append(f[-18:-9])
        #years.append(f[22:26])
        years.append(f[fname_left:fname_right])

    positions = [220, 900, 2100, 3500, 3600]
    positions_pipeline = ['pipe_220', 'pipe_900', 'pipe_2100', 'pipe_3500', 
                          'pipe_3600']
       
    # Condition to check if there is a year in the filename:
    if (years[0].isnumeric()) == True: 
        stdev_df = pd.DataFrame(std_pos, columns=positions, dtype = float, 
                                index=filenames)
        min_df = pd.DataFrame(min_pos, columns=positions, dtype = float, 
                                    index=filenames)
        max_df = pd.DataFrame(max_pos, columns=positions, dtype = float, 
                                    index=filenames)
        mean_df = pd.DataFrame(mean_pos, columns=positions, dtype = float, 
                                    index=filenames)
        median_df = pd.DataFrame(median_pos, columns=positions, dtype = float, 
                                    index=filenames)
        years = pd.DataFrame(years, columns=['Year'],index=filenames)
        stdev_df = pd.concat([stdev_df, years],1)
        min_df = pd.concat([min_df, years],1)
        max_df = pd.concat([max_df, years],1)
        mean_df = pd.concat([mean_df, years],1)
        median_df = pd.concat([median_df, years],1)
    else:
        stdev_df = pd.DataFrame(std_pos, columns=positions_pipeline, 
                                dtype = float, index=filenames)
        min_df = pd.DataFrame(min_pos, columns=positions_pipeline, 
                              dtype = float, index=filenames)
        max_df = pd.DataFrame(max_pos, columns=positions_pipeline, 
                              dtype = float, index=filenames)
        mean_df = pd.DataFrame(mean_pos, columns=positions_pipeline, 
                               dtype = float, index=filenames)
        median_df = pd.DataFrame(median_pos, columns=positions_pipeline, 
                                 dtype = float, index=filenames)
    
    #return stdev_df, min_df, max_df, mean_df, median_df
    if db_return == 'mean_df':
        return mean_df 
    if db_return == 'stdev_df':
        return stdev_df

In [ ]:
positions_list = [(220, 160), (900,700), (2100,1100), (3500,300), (3600,1700)]

mean_yearly = create_databases(updated_yearly, positions_list, fname_left, fname_right,'mean_df')
mean_pipeline = create_databases(science_pipeline, positions_list, fname_left, fname_right,'mean_df')

combine_by_file = pd.concat([mean_yearly,mean_pipeline],1)
mean_yearly

In [ ]:
#combine_by_file['220_norm'] = (combine_by_file[220] - combine_by_file['pipe_220']) / combine_by_file['pipe_220']
#combine_by_file['900_norm'] = (combine_by_file[900] - combine_by_file['pipe_900']) / combine_by_file['pipe_900']
#combine_by_file['2100_norm'] = (combine_by_file[2100] - combine_by_file['pipe_2100']) / combine_by_file['pipe_2100']
#combine_by_file['3500_norm'] = (combine_by_file[3500] - combine_by_file['pipe_3500']) / combine_by_file['pipe_3500']
#combine_by_file['3600_norm'] = (combine_by_file[3600] - combine_by_file['pipe_3600']) / combine_by_file['pipe_3600']

combine_by_file['220_diff'] = (combine_by_file[220] - combine_by_file['pipe_220'])
combine_by_file['900_diff'] = (combine_by_file[900] - combine_by_file['pipe_900']) 
combine_by_file['2100_diff'] = (combine_by_file[2100] - combine_by_file['pipe_2100'])
combine_by_file['3500_diff'] = (combine_by_file[3500] - combine_by_file['pipe_3500']) 
combine_by_file['3600_diff'] = (combine_by_file[3600] - combine_by_file['pipe_3600'])

mean_sorted_by_year_diff = combine_by_file.sort_values(by=['Year'])
mean_sorted_by_year_diff

In [ ]:
x = list([2012,2013,2014,2015,2016,2017,2018,2019,2020,2021])
#x2 = list([5,1,4,6,3,2])
x3 = list([1, 2, 3, 4, 5, 6])
y = list(mean_sorted_by_year['220_diff'].values)
y2 = list(mean_sorted_by_year['900_diff'].values)
y3 = list(mean_sorted_by_year['2100_diff'].values)
y4 = list(mean_sorted_by_year['3500_diff'].values)
y5 = list(mean_sorted_by_year['3600_diff'].values)

y12 = np.add(y,y2)
y34 = np.add(y3,y4)
y125 = np.add(y12,y5)
y12534 = np.add(y125,y34)
y_all = y12534/5


#mean of the 5 y over time, and standard dev upper limit and lower limit
#Increase in the standard dev over time 
figure(figsize=(12, 10))
#plt.plot(x,y_all, label='Mean of Standard Deviations of All Regions', linewidth=7.0, linestyle='dashed')
#plt.plot(x,y, label='Region 1')
#plt.plot(x,y2, label='Region 2')
#plt.plot(x,y3, label='Region 3')
#plt.plot(x,y4, label='Region 4')
plt.plot(x,y5, label='Region 1')
plt.axhline(linewidth=2, color='r', alpha=0.5)
plt.xlabel('Year of Science Data Observation', fontsize=20)
plt.ylabel(''r'$\sigma_{Original} $ - 'r'$\sigma_{Yearly}$', fontsize=30)
plt.title('Changes in Science Data Standard Deviation Differences ')

plt.legend()
plt.savefig('updated_changes_in_std_normalized_overtime.pdf') 

In [ ]:
positions_list = [(220, 160), (900,700), (2100,1100), (3500,300), (3600,1700)]

stdev_yearly = create_databases(updated_yearly, positions_list, fname_left, fname_right,'stdev_df')
stdev_pipeline = create_databases(science_pipeline, positions_list, fname_left, fname_right, 'stdev_df')

combine_by_file = pd.concat([stdev_yearly,stdev_pipeline],1)
stdev_yearly

In [ ]:
combine_by_file['220_norm'] = (combine_by_file[220] - combine_by_file['pipe_220']) / combine_by_file['pipe_220']
combine_by_file['900_norm'] = (combine_by_file[900] - combine_by_file['pipe_900']) / combine_by_file['pipe_900']
combine_by_file['2100_norm'] = (combine_by_file[2100] - combine_by_file['pipe_2100']) / combine_by_file['pipe_2100']
combine_by_file['3500_norm'] = (combine_by_file[3500] - combine_by_file['pipe_3500']) / combine_by_file['pipe_3500']
combine_by_file['3600_norm'] = (combine_by_file[3600] - combine_by_file['pipe_3600']) / combine_by_file['pipe_3600']

combine_by_file['220_diff'] = (combine_by_file[220] - combine_by_file['pipe_220'])
combine_by_file['900_diff'] = (combine_by_file[900] - combine_by_file['pipe_900']) 
combine_by_file['2100_diff'] = (combine_by_file[2100] - combine_by_file['pipe_2100'])
combine_by_file['3500_diff'] = (combine_by_file[3500] - combine_by_file['pipe_3500']) 
combine_by_file['3600_diff'] = (combine_by_file[3600] - combine_by_file['pipe_3600'])

stdev_sorted_by_year = combine_by_file.sort_values(by=['Year'])
stdev_sorted_by_year

In [ ]:
x = list([2012,2013,2014,2015,2016,2017,2018,2019,2020,2021])
#x2 = list([5,1,4,6,3,2])
x3 = list([1, 2, 3, 4, 5, 6])
y = list(stdev_sorted_by_year['220_norm'].values)
y2 = list(stdev_sorted_by_year['900_norm'].values)
y3 = list(stdev_sorted_by_year['2100_norm'].values)
y4 = list(stdev_sorted_by_year['3500_norm'].values)
y5 = list(stdev_sorted_by_year['3600_norm'].values)

y12 = np.add(y,y2)
y34 = np.add(y3,y4)
y125 = np.add(y12,y5)
y12534 = np.add(y125,y34)
y_all = y12534/5

#mean of the 5 y over time, and standard dev upper limit and lower limit
#Increase in the standard dev over time 
figure(figsize=(12, 10))
plt.plot(x,y_all, label='Mean of Standard Deviations of All Regions', linewidth=7.0, linestyle='dashed')
plt.plot(x,y, label='Region 1')
plt.plot(x,y2, label='Region 2')
plt.plot(x,y3, label='Region 3')
plt.plot(x,y4, label='Region 4')
plt.plot(x,y5, label='Region 5')
plt.axhline(linewidth=2, color='r', alpha=0.5)
plt.xlabel('Year of Science Data Observation', fontsize=20)
plt.ylabel(''r'$\sigma_{Original} $ - 'r'$\sigma_{Yearly}$', fontsize=30)
plt.title('Changes in Science Data Standard Deviation Differences ')

plt.legend()
plt.savefig('updated_changes_in_std_normalized_overtime.pdf') 

In [ ]:
image_biyearly = fits.getdata(biyearly_image,1)
image_curr = fits.getdata(current_image,1)
image_yearly = fits.getdata(yearly_image,1)

figure(figsize=(10, 8), dpi=80)

plt.hist(image_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=100, label='Bi-yearly Reference File',color='red')
plt.hist(image_curr.flatten(),alpha=0.5, range=(-20,20), bins=100, label='Current Reference File')
#plt.hist(image_yearly.flatten(),alpha=0.5, range=(-20,20), bins=100, label='Yearly Reference File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Fullframe Reference Images - Comparing Cadence')
plt.legend()
plt.savefig('fullframe_histogram_compare_biyearly_current_refs.pdf') 

In [ ]:
image_biyearly = fits.getdata(biyearly_image,1)
image_curr = fits.getdata(current_image,1)
image_yearly = fits.getdata(yearly_image,1)

figure(figsize=(10, 8), dpi=80)

#plt.hist(image_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=100, label='Bi-yearly Reference File')
plt.hist(image_curr.flatten(),alpha=0.5, range=(-20,20), bins=100, label='Original Reference File')
plt.hist(image_yearly.flatten(),alpha=0.5, range=(-20,20), bins=100, label='Yearly Reference File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Full-frame Science Data - Comparing Original vs. Yearly Refernce File')
plt.legend()
plt.savefig('fullframe_histogram_compare_refs.pdf') 

In [ ]:
position = (220,160)
size = (101, 101)

cutout_biyearly = Cutout2D(image_biyearly, position , size)
cutout1_biyearly = cutout_biyearly.data

cutout_yearly = Cutout2D(image_yearly, position , size)
cutout1_yearly = cutout_yearly.data

cutout_curr = Cutout2D(image_curr, position , size)
cutout1_curr = cutout_curr.data

figure(figsize=(10, 8), dpi=80)

#plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Bi-yearly Reference File')
plt.hist(cutout1_curr.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Original Reference File')
plt.hist(cutout1_yearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Yearly Reference File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Region 1 Science Data - Comparing Original vs. Yearly Reference File')
plt.legend()
plt.savefig('region1_histogram_compare_refs.pdf') 

In [ ]:
position = (900,700)
size = (101, 101)

cutout_biyearly = Cutout2D(image_biyearly, position , size)
cutout1_biyearly = cutout_biyearly.data

cutout_yearly = Cutout2D(image_yearly, position , size)
cutout1_yearly = cutout_yearly.data

cutout_curr = Cutout2D(image_curr, position , size)
cutout1_curr = cutout_curr.data

figure(figsize=(10, 8), dpi=80)

#plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Bi-yearly Reference File')
plt.hist(cutout1_curr.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Original Reference File')
plt.hist(cutout1_yearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Yearly Reference File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Region 2 Science Data - Comparing Original vs. Yearly Reference File')
plt.legend()
plt.savefig('region2_histogram_compare_refs.pdf') 

In [ ]:
position = (2100,1100)
size = (101, 101)

cutout_biyearly = Cutout2D(image_biyearly, position , size)
cutout1_biyearly = cutout_biyearly.data

cutout_yearly = Cutout2D(image_yearly, position , size)
cutout1_yearly = cutout_yearly.data

cutout_curr = Cutout2D(image_curr, position , size)
cutout1_curr = cutout_curr.data

figure(figsize=(10, 8), dpi=80)

#plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Bi-yearly Reference File')
plt.hist(cutout1_curr.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Original Reference File')
plt.hist(cutout1_yearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Yearly Reference File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Region 3 Science Data - Comparing Original vs. Yearly Reference File')
plt.legend()
plt.savefig('region3_histogram_compare_refs.pdf') 

In [ ]:
position = (3500,300)
size = (101, 101)

cutout_biyearly = Cutout2D(image_biyearly, position , size)
cutout1_biyearly = cutout_biyearly.data

cutout_yearly = Cutout2D(image_yearly, position , size)
cutout1_yearly = cutout_yearly.data

cutout_curr = Cutout2D(image_curr, position , size)
cutout1_curr = cutout_curr.data

figure(figsize=(10, 8), dpi=80)

#plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Bi-yearly Reference File')
plt.hist(cutout1_curr.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Original Reference File')
plt.hist(cutout1_yearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Yearly Reference File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Region 4 Science Data - Comparing Original vs. Yearly Reference File')
plt.legend()
plt.savefig('region4_histogram_compare_refs.pdf') 

In [ ]:
position = (3500,300)
size = (101, 101)

cutout_biyearly = Cutout2D(image_biyearly, position , size)
cutout1_biyearly = cutout_biyearly.data

cutout_yearly = Cutout2D(image_yearly, position , size)
cutout1_yearly = cutout_yearly.data

cutout_curr = Cutout2D(image_curr, position , size)
cutout1_curr = cutout_curr.data

diff_cutout = cutout1_yearly - cutout1_curr

figure(figsize=(10, 8), dpi=80)

#plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Bi-yearly Reference File')
plt.hist(diff_cutout.flatten(),alpha=0.5, range=(-0.5,0.5), bins=200, label='Yearly Ref File - Original Ref File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Region 4 Science Data Difference - Comparing Original vs. Yearly Reference File')
plt.legend()
plt.savefig('difference_histo_region4_compare_refs.pdf') 

In [ ]:
position = (3600,1700)
size = (101, 101)

cutout_biyearly = Cutout2D(image_biyearly, position , size)
cutout1_biyearly = cutout_biyearly.data

cutout_yearly = Cutout2D(image_yearly, position , size)
cutout1_yearly = cutout_yearly.data

cutout_curr = Cutout2D(image_curr, position , size)
cutout1_curr = cutout_curr.data

diff_cutout = cutout1_yearly - cutout1_curr

figure(figsize=(10, 8), dpi=80)

#plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Bi-yearly Reference File')
plt.hist(diff_cutout.flatten(),alpha=0.5, range=(-0.5,0.5), bins=200, label='Yearly Ref File - Original Ref File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Region 5 Science Data Difference - Comparing Original vs. Yearly Reference File')
plt.legend()
plt.savefig('difference_histo_region5_compare_refs.pdf') 

In [ ]:
position = (3600,1700)
size = (101, 101)

cutout_biyearly = Cutout2D(image_biyearly, position , size)
cutout1_biyearly = cutout_biyearly.data

cutout_yearly = Cutout2D(image_yearly, position , size)
cutout1_yearly = cutout_yearly.data

cutout_curr = Cutout2D(image_curr, position , size)
cutout1_curr = cutout_curr.data

figure(figsize=(10, 8), dpi=80)

#plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Bi-yearly Reference File')
plt.hist(cutout1_curr.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Original Reference File')
plt.hist(cutout1_yearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Yearly Reference File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Region 5 Science Data - Comparing Original vs. Yearly Reference File')
plt.legend()
plt.savefig('region5_histogram_compare_refs.pdf') 

In [ ]:
def create_databases(input_data, positions_list, fname_left, fname_right):
    size = (101, 101)
    
    min_pos = []
    max_pos = []
    mean_pos = []
    median_pos = []
    std_pos = []
    
    for f in input_data: 
        image_data = fits.getdata(f,1)
        means = []
        maxes = []
        mins = []
        medians = []
        stdevs = []
        for p in positions_list: 
            cutout = Cutout2D(image_data, p, size)
            cutout1 = cutout.data
            mins.append(np.min(cutout1))
            maxes.append(np.max(cutout1))
            means.append(np.mean(cutout1))
            medians.append(np.median(cutout1))
            stdevs.append(np.std(cutout1))
        std_pos.append(stdevs)
        min_pos.append(mins)
        max_pos.append(maxes)
        mean_pos.append(means)
        median_pos.append(medians)
    filenames = []
    years = []

    for f in input_data:
        filenames.append(f[-18:-9])
        years.append(f[fname_left:fname_right])

    positions = ['stdev_220', 'stdev_900', 'stdev_2100', 'stdev_3500', 
                 'stdev_3600']
    positions_min = ['min_220', 'min_900', 'min_2100', 'min_3500', 
                          'min_3600']
    positions_max = ['max_220', 'max_900', 'max_2100', 'max_3500', 
                          'max_3600']
    positions_mean = ['mean_220', 'mean_900', 'mean_2100', 'mean_3500', 
                          'mean_3600']
    positions_med = ['med_220', 'med_900', 'med_2100', 'med_3500', 
                          'med_3600']
    positions_pipeline = ['pipe_220', 'pipe_900', 'pipe_2100', 'pipe_3500', 
                          'pipe_3600']
    pos_minpipe = ['min_pipe_220', 'min_pipe_900', 'min_pipe_2100', 
                   'min_pipe_3500', 'min_pipe_3600']
    pos_maxpipe = ['max_pipe_220', 'max_pipe_900', 'max_pipe_2100', 
                   'max_pipe_3500', 'max_pipe_3600']
    pos_meanpipe = ['mean_pipe_220', 'mean_pipe_900', 'mean_pipe_2100', 
                    'mean_pipe_3500', 'mean_pipe_3600']
    pos_medpipe = ['mean_pipe_220', 'mean_pipe_900', 'mean_pipe_2100', 
                   'mean_pipe_3500', 'mean_pipe_3600']
       
    # Condition to check if there is a year in the filename:
    if (years[0].isnumeric()) == True: 
        stdev_df = pd.DataFrame(std_pos, columns=positions, dtype = float, 
                                index=filenames)
        min_df = pd.DataFrame(min_pos, columns=positions_min, dtype = float, 
                                    index=filenames)
        max_df = pd.DataFrame(max_pos, columns=positions_max, dtype = float, 
                                    index=filenames)
        mean_df = pd.DataFrame(mean_pos, columns=positions_mean, dtype = float, 
                                    index=filenames)
        median_df = pd.DataFrame(median_pos, columns=positions_med, dtype = float, 
                                    index=filenames)
        years = pd.DataFrame(years,columns=['Year'], index=filenames)
        stdev_df = pd.concat([stdev_df, years],1)
        min_df = pd.concat([min_df, years],1)
        max_df = pd.concat([max_df, years],1)
        mean_df = pd.concat([mean_df, years],1)
        median_df = pd.concat([median_df, years],1)
    else:
        stdev_df = pd.DataFrame(std_pos, columns=positions_pipeline, 
                                dtype = float, index=filenames)
        min_df = pd.DataFrame(min_pos, columns=pos_minpipe, 
                              dtype = float, index=filenames)
        max_df = pd.DataFrame(max_pos, columns=pos_maxpipe, 
                              dtype = float, index=filenames)
        mean_df = pd.DataFrame(mean_pos, columns=pos_meanpipe, 
                               dtype = float, index=filenames)
        median_df = pd.DataFrame(median_pos, columns=pos_medpipe, 
                                 dtype = float, index=filenames)
    
    return stdev_df, min_df, max_df, mean_df, median_df

In [ ]:
positions_list = [(220, 160), (900,700), (2100,1100), (3500,300), (3600,1700)]

stdev_yearly, min_yearly, max_yearly, mean_yearly, median_yearly = create_databases(updated_yearly, positions_list, fname_left, fname_right)
yearly_stats = pd.concat([stdev_yearly,min_yearly, max_yearly, mean_yearly, median_yearly],1)



In [ ]:
yearly_stats = yearly_stats.T.drop_duplicates().T
yearly_stats = yearly_stats.sort_values(by=['Year'])
yearly_stats

curr_stats = curr_stats.T.drop_duplicates().T
#curr_stats = curr_stats.sort_values(by=['Year'])

In [ ]:
yearly_stats_table = yearly_stats.T
yearly_stats_table.index
ordered_yearly_stats = yearly_stats_table.reindex(['Year', 
                                                  'stdev_220', 'stdev_900', 'stdev_2100', 'stdev_3500', 'stdev_3600',
                                                  'min_220', 'min_900', 'min_2100', 'min_3500', 'min_3600',
                                                  'max_220', 'max_900', 'max_2100', 'max_3500', 'max_3600', 
                                                  'mean_220','mean_900', 'mean_2100', 'mean_3500', 'mean_3600',
                                                  'med_220', 'med_900','med_2100', 'med_3500', 'med_3600'])
ordered_yearly_stats.rename(index={'stdev_220': 'stdev Region 1','stdev_900': 'Region 2', 'stdev_2100': 'Region 3',
                                  'stdev_3500': 'Region 4','stdev_3600': 'Region 5', 'min_220': 'min Region 1',
                                   'min_900': 'Region 2', 'min_2100': 'Region 3',
                                  'min_3500': 'Region 4','min_3600': 'Region 5','max_220': 'max Region 1',
                                   'max_900': 'Region 2', 'max_2100': 'Region 3',
                                  'max_3500': 'Region 4','max_3600': 'Region 5','mean_220': 'mean Region 1',
                                   'mean_900': 'Region 2', 'mean_2100': 'Region 3',
                                  'mean_3500': 'Region 4','mean_3600': 'Region 5',
                                   'med_220': 'med Region 1','med_900': 'Region 2', 'med_2100': 'Region 3',
                                  'med_3500': 'Region 4','med_3600': 'Region 5'},inplace=True)


In [ ]:
for x in ordered_yearly_stats: 
    y = list(ordered_yearly_stats[x].values)
    print (x)
    stdev = np.mean(y[1:6])
    minim = np.mean(y[7:11])
    maximum = np.mean(y[11:16])
    mean = np.mean(y[16:21])
    med = np.mean(y[21:])
    
    print (stdev)
    #print(minim)
    #print(maximum)
    print(mean)
    print(med)

ordered_yearly_stats

In [ ]:
positions_list = [(220, 160), (900,700), (2100,1100), (3500,300), (3600,1700)]

stdev_yearly, min_yearly, max_yearly, mean_yearly, median_yearly = create_databases(updated_yearly, positions_list, fname_left, fname_right)
yearly_stats = pd.concat([stdev_yearly, mean_yearly],1)
yearly_stats

stdev_curr, min_curr, max_curr, mean_curr, median_curr = create_databases(science_pipeline, positions_list, fname_left, fname_right)
curr_stats = pd.concat([stdev_curr, mean_curr],1)

curr_and_yearly_stats = pd.concat([yearly_stats, curr_stats],1)

curr_and_yearly_stats = curr_and_yearly_stats.T.drop_duplicates().T
curr_and_yearly_stats = curr_and_yearly_stats.sort_values(by=['Year'])
curr_and_yearly_stats

In [ ]:
differences_curr_and_yearly = pd.DataFrame()

differences_curr_and_yearly['220_diff_mean'] = (curr_and_yearly_stats['mean_220'] - curr_and_yearly_stats['mean_pipe_220'])
differences_curr_and_yearly['900_diff_mean'] = (curr_and_yearly_stats['mean_900'] - curr_and_yearly_stats['mean_pipe_900']) 
differences_curr_and_yearly['2100_diff_mean'] = (curr_and_yearly_stats['mean_2100'] - curr_and_yearly_stats['mean_pipe_2100'])
differences_curr_and_yearly['3500_diff_mean'] = (curr_and_yearly_stats['mean_3500'] - curr_and_yearly_stats['mean_pipe_3500']) 
differences_curr_and_yearly['3600_diff_mean'] = (curr_and_yearly_stats['mean_3600'] - curr_and_yearly_stats['mean_pipe_3600'])

differences_curr_and_yearly['220_diff_stdev'] = (curr_and_yearly_stats['stdev_220'] - curr_and_yearly_stats['pipe_220'])
differences_curr_and_yearly['900_diff_stdev'] = (curr_and_yearly_stats['stdev_900'] - curr_and_yearly_stats['pipe_900']) 
differences_curr_and_yearly['2100_diff_stdev'] = (curr_and_yearly_stats['stdev_2100'] - curr_and_yearly_stats['pipe_2100'])
differences_curr_and_yearly['3500_diff_stdev'] = (curr_and_yearly_stats['stdev_3500'] - curr_and_yearly_stats['pipe_3500']) 
differences_curr_and_yearly['3600_diff_stdev'] = (curr_and_yearly_stats['stdev_3600'] - curr_and_yearly_stats['pipe_3600'])
differences_curr_and_yearly['Year'] = curr_and_yearly_stats['Year']

differences_curr_and_yearly
differences_curr_and_yearly = differences_curr_and_yearly.T
differences_curr_and_yearly.index

ordered_differences_curr_and_yearly = differences_curr_and_yearly.reindex(['Year',
                                                  '220_diff_mean', '900_diff_mean', '2100_diff_mean', '3500_diff_mean', '3600_diff_mean',
                                                  '220_diff_stdev', '900_diff_stdev', '2100_diff_stdev', '3500_diff_stdev', '3600_diff_stdev'])
ordered_differences_curr_and_yearly

In [ ]:
differences_curr_and_yearly = differences_curr_and_yearly.T
differences_curr_and_yearly.index
ordered_differences_curr_and_yearly = differences_curr_and_yearly.reindex(['Year',
                                                  '220_diff_mean', '900_diff_mean', '2100_diff_mean', '3500_diff_mean', '3600_diff_mean',
                                                  '220_diff_stdev', '900_diff_stdev', '2100_diff_stdev', '3500_diff_stdev', '3600_diff_stdev'])
ordered_differences_curr_and_yearly.rename(index={'220_diff_stdev': 'Differences in Std Dev: Region 1',
                                                  '900_diff_stdev': 'Region 2', 
                                                  '2100_diff_stdev': 'Region 3',
                                                  '3500_diff_stdev': 'Region 4',
                                                  '3600_diff_stdev': 'Region 5', 
                                                  '220_diff_mean': 'Differences in Mean: Region 1',
                                                  '900_diff_mean': 'Region 2', 
                                                  '2100_diff_mean': 'Region 3',
                                                  '3500_diff_mean': 'Region 4',
                                                  '3600_diff_mean': 'Region 5'},inplace=True)

ordered_differences_curr_and_yearly 

In [ ]:
for x in ordered_differences_curr_and_yearly: 
    y = list(ordered_differences_curr_and_yearly[x].values)
    print (x)
    mean = np.mean(y[1:6])
    stdev = np.mean(y[7:12])
    print (stdev)
    print(mean)

In [ ]:
position = (3500,300)
size = (101, 101)

cutout_biyearly = Cutout2D(image_biyearly, position , size)
cutout1_biyearly = cutout_biyearly.data

cutout_yearly = Cutout2D(image_yearly, position , size)
cutout1_yearly = cutout_yearly.data

cutout_curr = Cutout2D(image_curr, position , size)
cutout1_curr = cutout_curr.data

figure(figsize=(10, 8), dpi=80)

plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Bi-yearly Reference File')
plt.hist(cutout1_curr.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Original Reference File')
#plt.hist(cutout1_yearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Yearly Reference File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Region 4 Science Data - Comparing Original vs. Biyearly Reference File')
plt.legend()
plt.savefig('region4_biyearly_histogram_compare_refs.pdf') 

In [ ]:
image_curr_2013 = fits.getdata(current_image_2013,1)
image_yearly_2013 = fits.getdata(yearly_image_2013,1)

position4 = (3500,300)
size = (101, 101)

figure(figsize=(10, 8), dpi=80)

cutout_yearly_4 = Cutout2D(image_yearly_2013, position4 , size)
cutout1_yearly_4 = cutout_yearly_4.data

cutout_curr_4 = Cutout2D(image_curr_2013, position4 , size)
cutout1_curr_4 = cutout_curr_4.data

diff_cutout_4 = cutout1_yearly_4 - cutout1_curr_4

position5 = (3500,300)

cutout_yearly_5 = Cutout2D(image_yearly_2013, position5, size)
cutout1_yearly_5 = cutout_yearly_5.data

cutout_curr_5 = Cutout2D(image_curr_2013, position5, size)
cutout1_curr_5 = cutout_curr_5.data

diff_cutout_5 = cutout1_yearly_5 - cutout1_curr_5

#plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Bi-yearly Reference File')
#plt.hist(diff_cutout_4.flatten(),alpha=0.5, range=(-0.5,0.5), bins=200, label='Region 4 - 2019')
plt.hist(diff_cutout_5.flatten(),alpha=0.5, range=(-0.5,0.5), bins=200, label='Region 4 - 2013')
plt.xlabel('Difference in Counts')
plt.ylabel('Number of Pixels')
plt.title('Region 4 Science Data Difference - Comparing Yearly vs. Original Reference File')
plt.legend()
plt.savefig('difference_histo_region4_2013_2019_compare_refs.pdf') 

In [ ]:
position = (3600,1700)
size = (101, 101)

cutout_biyearly = Cutout2D(image_biyearly, position , size)
cutout1_biyearly = cutout_biyearly.data

cutout_yearly = Cutout2D(image_yearly, position , size)
cutout1_yearly = cutout_yearly.data

cutout_curr = Cutout2D(image_curr, position , size)
cutout1_curr = cutout_curr.data

figure(figsize=(10, 8), dpi=80)

plt.hist(cutout1_biyearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Bi-yearly Reference File')
plt.hist(cutout1_curr.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Original Reference File')
#plt.hist(cutout1_yearly.flatten(),alpha=0.5, range=(-20,20), bins=200, label='Yearly Reference File')
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.title('Region 5 Science Data - Comparing Original vs. Biyearly Reference File')
plt.legend()
plt.savefig('region5_biyearly_histogram_compare_refs.pdf') 

In [ ]:
current_A_low_curr = fits.getdata('../../2022_data/16819058i_fls.fits',1)
ref_A_low_2021 = fits.getdata('../../2022_data/deliverable_reference_files/2021_pf_A_low_curr.fits',1)
ref_A_low_2017 = fits.getdata('../../2022_data/deliverable_reference_files/2017_pf_A_low_curr.fits',1)
ref_A_low_2013 = fits.getdata('../../2022_data/deliverable_reference_files/2013_pf_A_low_curr.fits',1)
ref_A_low_2012 = fits.getdata('../../2022_data/deliverable_reference_files/2012_pf_A_low_curr.fits',1)
ref_A_low_2014 = fits.getdata('../../2022_data/deliverable_reference_files/2014_pf_A_low_curr.fits',1)
ref_A_low_2015 = fits.getdata('../../2022_data/deliverable_reference_files/2015_pf_A_low_curr.fits',1)
ref_A_low_2016 = fits.getdata('../../2022_data/deliverable_reference_files/2016_pf_A_low_curr.fits',1)
ref_A_low_2018 = fits.getdata('../../2022_data/deliverable_reference_files/2018_pf_A_low_curr.fits',1)
ref_A_low_2019 = fits.getdata('../../2022_data/deliverable_reference_files/2019_pf_A_low_curr.fits',1)
ref_A_low_2020 = fits.getdata('../../2022_data/deliverable_reference_files/2020_pf_A_low_curr.fits',1)

figure(figsize=(10, 8), dpi=80)

plt.hist(ref_A_low_2021.flatten(),alpha=0.5, range=(2,3.2), bins=500, label='2021 Reference File',color='red')
#plt.hist(ref_A_low_2019.flatten(),alpha=0.5, range=(2,3.2), bins=500, label='2019 Reference File')
plt.hist(ref_A_low_2017.flatten(),alpha=0.5, range=(2,3.2), bins=500, label='2017 Reference File')
#plt.hist(ref_A_low_2015.flatten(),alpha=0.5, range=(2.0,3.2), bins=500, label='2015 Reference File')
#plt.hist(ref_A_low_2013.flatten(),alpha=0.5, range=(2,3.2), bins=500, label='2013 Reference File')
plt.hist(current_A_low_curr.flatten(),alpha=0.5, range=(2.0,3.2), bins=500, label='Original Reference File')
plt.xlabel('Counts(e-)/Second')
plt.ylabel('Number of Pixels')
plt.title('Comparing Original and New Time-Dependent Reference Files (UVIS2)')
plt.legend()
plt.savefig('uvis2_original_vs_time_dependent_reference_file_low_current.pdf') 

In [ ]:
current_A_low_curr = fits.getdata('../../2022_data/16819058i_fls.fits',1)
biyearly_2020_2021 = fits.getdata('../../2022_data/2020_2021_A_final_cal.fits',1)
ref_A_low_2021 = fits.getdata('../../2022_data/deliverable_reference_files/2021_pf_A_low_curr.fits',1)
ref_A_low_2017 = fits.getdata('../../2022_data/deliverable_reference_files/2017_pf_A_low_curr.fits',1)

figure(figsize=(10, 8), dpi=80)

plt.hist(ref_A_low_2021.flatten(),alpha=0.5, range=(1,3.5), bins=500, label='2021 Reference File',color='red')
#plt.hist(ref_A_low_2017.flatten(),alpha=0.5, range=(1,3.5), bins=500, label='2017 Reference File')
plt.hist(biyearly_2020_2021.flatten(),alpha=0.5, range=(1,3.5), bins=500, label='2020-2021 biyearly')                                  
#plt.hist(current_A_low_curr.flatten(),alpha=0.5, range=(1,3.5), bins=500, label='Current Reference File')
plt.xlabel('Counts (e-)/Second')
plt.ylabel('Number of Pixels')
plt.title('Comparing Yearly versus Biyearly Time-Dependent Reference Files (UVIS2)')
plt.legend()
plt.savefig('uvis2_biyearly_vs_time_dependent_reference_file_low_current.pdf')

In [ ]:
current_A_low_curr = fits.getdata('../../2022_data/16819058i_fls.fits',1)
biyearly_2020_2021 = fits.getdata('../../2022_data/2020_2021_A_final_cal.fits',1)
ref_A_low_2021 = fits.getdata('../../2022_data/deliverable_reference_files/2021_pf_A_low_curr.fits',1)

figure(figsize=(10, 8), dpi=80)

diff = ref_A_low_2021.flatten() - biyearly_2020_2021.flatten()

print(type(diff))

diff[diff != 0]
print(np.unique(diff))


plt.hist(diff,alpha=0.5, range=(-0.1,0.1), bins=500, label='2021 Reference File - Biyearly Reference File',color='red')
#plt.hist(ref_A_low_2017.flatten(),alpha=0.5, range=(1,3.5), bins=500, label='2017 Reference File')
#plt.hist(biyearly_2020_2021.flatten(),alpha=0.5, range=(1,3.5), bins=500, label='2020-2021 biyearly')                                  
#plt.hist(current_A_low_curr.flatten(),alpha=0.5, range=(1,3.5), bins=500, label='Current Reference File')
plt.xlabel('Counts (e-)/Second')
plt.ylabel('Number of Pixels')
plt.title('Comparing Yearly versus Biyearly Time-Dependent Reference Files (UVIS2)')
plt.legend()
plt.savefig('uvis2_yearly_minus_biyearly_time_dependent_reference_file_low_current.pdf')

In [ ]:
positions_list = [(220, 160), (900,700), (2100,1100), (3500,300), (3600,1700)]

stdev_yearly, min_yearly, max_yearly, mean_yearly, median_yearly = create_databases(updated_yearly, positions_list, fname_left, fname_right)
yearly_stats = pd.concat([stdev_yearly,min_yearly, max_yearly, mean_yearly, median_yearly],1)

In [ ]:
current_A_low_curr = fits.getdata('../../2022_data/16819058i_fls.fits',1)
ref_A_low_2021 = fits.getdata('../../2022_data/deliverable_reference_files/2021_pf_A_low_curr.fits',1)

position4 = (3500,300)
size = (101, 101)

figure(figsize=(10, 8), dpi=80)

cutout_yearly_4 = Cutout2D(ref_A_low_2021, position4 , size)
cutout1_yearly_4 = cutout_yearly_4.data

cutout_curr_4 = Cutout2D(current_A_low_curr, position4 , size)
cutout1_curr_4 = cutout_curr_4.data

diff_cutout_4 = cutout1_curr_4 - cutout1_yearly_4

position5 = (3600,1700)

cutout_yearly_5 = Cutout2D(ref_A_low_2021, position5, size)
cutout1_yearly_5 = cutout_yearly_5.data

cutout_curr_5 = Cutout2D(current_A_low_curr, position5, size)
cutout1_curr_5 = cutout_curr_5.data

diff_cutout_5 = cutout1_curr_5 - cutout1_yearly_5

plt.hist(diff_cutout_4.flatten(),alpha=0.5, range=(-0.01,0.1), bins=200, label='Region 4 - 2021 Reference File - Original Ref File')
plt.hist(diff_cutout_5.flatten(),alpha=0.5, range=(-0.01,0.1), bins=200, label='Region 5 - 2021 Reference File - Original Ref File')
plt.xlabel('Difference in Counts')
plt.ylabel('Number of Pixels')
plt.title('Reference File Region Difference - Comparing 2021 vs. Original Reference File')
plt.legend()
plt.savefig('diff_histo_region4_5_2021_reference_files.pdf') 

In [ ]:
current_A_low_curr = fits.getdata('../../2022_data/16819058i_fls.fits',1)
ref_A_low_2021 = fits.getdata('../../2022_data/deliverable_reference_files/2021_pf_A_low_curr.fits',1)

figure(figsize=(10, 8), dpi=80)

diff = current_A_low_curr - ref_A_low_2021

plt.hist(diff.flatten(),alpha=0.5, range=(-0.1,0.1), bins=200, label='Original Ref File - 2021 Reference File')
plt.xlabel('Difference in Counts')
plt.ylabel('Number of Pixels')
plt.title('(Original - 2021) Reference File Difference')
plt.legend()
plt.savefig('diff_histo_2021_reference_files.pdf') 

In [ ]:
current_A_low_curr = fits.getdata('../../2022_data/16819058i_fls.fits',1)
ref_A_low_2013 = fits.getdata('../../2022_data/deliverable_reference_files/2013_pf_A_low_curr.fits',1)

figure(figsize=(10, 8), dpi=80)

diff = current_A_low_curr - ref_A_low_2013

plt.hist(diff.flatten(),alpha=0.5, range=(-0.1,0.1), bins=200, label='Original Ref File - 2013 Reference File')
plt.xlabel('Difference in Counts')
plt.ylabel('Number of Pixels')
plt.title('(Original - 2013) Reference File Difference')
plt.legend()
plt.savefig('diff_histo_2013_reference_files.pdf') 

In [ ]:
current_A_low_curr = fits.getdata('../../2022_data/16819058i_fls.fits',1)
ref_A_low_2021 = fits.getdata('../../2022_data/deliverable_reference_files/2021_pf_A_low_curr.fits',1)
ref_A_low_2017 = fits.getdata('../../2022_data/deliverable_reference_files/2017_pf_A_low_curr.fits',1)
ref_A_low_2013 = fits.getdata('../../2022_data/deliverable_reference_files/2013_pf_A_low_curr.fits',1)
ref_A_low_2012 = fits.getdata('../../2022_data/deliverable_reference_files/2012_pf_A_low_curr.fits',1)
ref_A_low_2014 = fits.getdata('../../2022_data/deliverable_reference_files/2014_pf_A_low_curr.fits',1)
ref_A_low_2015 = fits.getdata('../../2022_data/deliverable_reference_files/2015_pf_A_low_curr.fits',1)
ref_A_low_2016 = fits.getdata('../../2022_data/deliverable_reference_files/2016_pf_A_low_curr.fits',1)
ref_A_low_2018 = fits.getdata('../../2022_data/deliverable_reference_files/2018_pf_A_low_curr.fits',1)
ref_A_low_2019 = fits.getdata('../../2022_data/deliverable_reference_files/2019_pf_A_low_curr.fits',1)
ref_A_low_2020 = fits.getdata('../../2022_data/deliverable_reference_files/2020_pf_A_low_curr.fits',1)

figure(figsize=(10, 8), dpi=80)

#plt.hist(ref_A_low_2021.flatten(),alpha=0.5, range=(2,3.2), bins=500, label='2021 Reference File',color='red')
#plt.hist(ref_A_low_2019.flatten(),alpha=0.5, range=(2,3.2), bins=500, label='2019 Reference File')
#plt.hist(ref_A_low_2017.flatten(),alpha=0.5, range=(2,3.2), bins=500, label='2017 Reference File')
#plt.hist(ref_A_low_2015.flatten(),alpha=0.5, range=(2.0,3.2), bins=500, label='2015 Reference File')
plt.hist(ref_A_low_2020.flatten(),alpha=0.5, range=(2,3.2), bins=500, label='2020 Reference File')
#plt.hist(ref_A_low_2013.flatten(),alpha=0.5, range=(2,3.2), bins=500, label='2013 Reference File')
plt.hist(current_A_low_curr.flatten(),alpha=0.3, range=(2.0,3.2), bins=500, label='Original Reference File')
plt.xlabel('Counts(e-)/Second')
plt.ylabel('Number of Pixels')
plt.title('Comparing Original and New Time-Dependent Reference Files (UVIS2)')
plt.legend()
plt.savefig('2020_uvis2_current_vs_time_dependent_reference_file_low_current.pdf')

In [ ]:
current_A_low_curr = fits.getdata('../../2022_data/16819058i_fls.fits',1)
ref_A_low_2013 = fits.getdata('../../2022_data/deliverable_reference_files/2013_pf_A_low_curr.fits',1)

figure(figsize=(10, 8), dpi=80)

diff = current_A_low_curr - ref_A_low_2013

plt.hist(diff.flatten(),alpha=0.5, range=(-0.1,0.1), bins=200, label='Original Ref File - 2013 Reference File')
plt.xlabel('Difference in Counts')
plt.ylabel('Number of Pixels')
plt.title('(Original - 2013) Reference File Difference')
plt.legend()
plt.savefig('diff_histo_2013_reference_files.pdf') 

In [ ]:
current_A_low_curr = fits.getdata('../../2022_data/16819058i_fls.fits',1)
ref_A_low_2020 = fits.getdata('../../2022_data/deliverable_reference_files/2020_pf_A_low_curr.fits',1)

figure(figsize=(10, 8), dpi=80)

diff = current_A_low_curr - ref_A_low_2020

plt.hist(diff.flatten(),alpha=0.5, range=(-0.1,0.1), bins=200, label='Original Ref File - 2020 Reference File')
plt.xlabel('Difference in Counts')
plt.ylabel('Number of Pixels')
plt.title('(Original - 2020) Reference File Difference')
plt.legend()
plt.savefig('diff_histo_2020_reference_files.pdf') 

In [ ]:
np.mean(current_A_low_curr)

In [ ]:
np.mean(ref_A_low_2020)

In [ ]:
biyearly_ref = fits.getdata('../../2022_data/2020_2021_A_final_cal.fits',1)
ref_A_low_2020 = fits.getdata('../../2022_data/deliverable_reference_files/2020_pf_A_low_curr.fits',1)

figure(figsize=(10, 8), dpi=80)

plt.hist(biyearly_ref.flatten(), alpha=0.5, range=(2,3.21), bins=500, label='Biyearly Reference File')
plt.hist(ref_A_low_2020.flatten(), alpha=0.5, range=(2,3.21), bins=500, label='Yearly Reference File')
plt.xlabel('Counts (e-)/Second')
plt.ylabel('Number of Pixels')
plt.title('Comparing Yearly to Biyearly Reference Files')
plt.legend()
plt.savefig('Better_version_Feb_2023_biyearly.pdf') 

In [ ]:
figure(figsize=(10, 8), dpi=80)
plt.hist(biyearly_ref.flatten(), alpha=0.5, range=(2.9,3.25), bins=500, label='Biyearly Reference File')
plt.hist(ref_A_low_2020.flatten(), alpha=0.5, range=(2.9,3.25), bins=500, label='Yearly Reference File')
plt.xlabel('Counts (e-)/Second')
plt.ylabel('Number of Pixels')
plt.title('Zoomed In: Comparing Yearly to Biyearly Reference Files')
plt.legend()
plt.savefig('zoom_in_biyearly.pdf') 

In [ ]:
biyear = biyearly_ref.flatten()
year = ref_A_low_2020.flatten()

print(np.mean(biyear), np.std(biyear))
print(np.mean(year), np.std(year))